# The Machine Learning Workflow

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


### Load Data

In [2]:
df = pd.read_csv('../data/RtmSimulation_kickstart.csv', index_col= 0)

In [ ]:
# TODO: visualize the data, print summary stats

#### Define target and features

In [3]:
y = df['lai']
X = df.iloc[:,1:13]
X_sentinel = df.iloc[:,3:13]
X_species_sentinel = df.iloc[:,2:13]
X_wetness_sentinel = df.iloc[:,[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

In [4]:
# select input data
X = X_wetness_sentinel

In [5]:
X.head(), y.head()

(    wetness  Sentinel_2A_492.4  Sentinel_2A_559.8  Sentinel_2A_664.6  \
 id                                                                     
 1      0.36           0.062092           0.131702           0.043197   
 2      0.47           0.052807           0.129987           0.043061   
 3      0.68           0.047937           0.139421           0.045780   
 4      0.80           0.045907           0.107761           0.033984   
 5      0.48           0.051712           0.136293           0.041502   
 
     Sentinel_2A_704.1  Sentinel_2A_740.5  Sentinel_2A_782.8  \
 id                                                            
 1            0.177134           0.401750           0.458003   
 2            0.153641           0.407523           0.466853   
 3            0.157121           0.395428           0.441620   
 4            0.128237           0.341315           0.385277   
 5            0.167564           0.407460           0.454137   
 
     Sentinel_2A_832.8  Sentinel_2A_8

#### Train-Test Split

In [6]:
# split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

# val set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True, stratify=y_train)


## Feature Engineering

### All features to numerical and Missing Values

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Identifying categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Creating a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])


X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
X_val = preprocessor.transform(X_val)


### Remove Outliers

In [ ]:
# TODO

### Transformations

In [8]:
# exp transfromation concat with original data
X_train_exp = np.exp(X_train)
X_test_exp = np.exp(X_test)
X_val_exp = np.exp(X_val)
X_train_exp = np.concatenate((X_train, X_train_exp), axis = 1)
X_test_exp = np.concatenate((X_test, X_test_exp), axis = 1)
X_val_exp = np.concatenate((X_val, X_val_exp), axis = 1)


# standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

# normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)
X_val_norm = scaler.transform(X_val)

# standardization + exp concat
scaler = StandardScaler()
scaler.fit(X_train_exp)
X_train_scaled_exp = scaler.transform(X_train_exp)
X_test_scaled_exp = scaler.transform(X_test_exp)
X_val_scaled_exp = scaler.transform(X_val_exp)



In [35]:
# select standardized input data
X_train = X_train_scaled
X_test = X_test_scaled
X_val = X_val_scaled



In [36]:
X_val

array([[ 0.08773349, -0.28812957, -0.46299568, ...,  0.37337968,
         0.41746805,  0.20413506],
       [-0.36026731,  0.1068488 ,  0.11779769, ...,  0.44589759,
        -0.28264652, -0.51328595],
       [ 0.13751136, -0.06079593, -0.01042041, ..., -0.13931193,
        -0.06634946, -0.21079655],
       ...,
       [-1.55493609, -0.26706107, -0.07488734, ...,  0.75063173,
        -0.19631258, -0.18560636],
       [-0.45982304, -0.18344231, -0.15851261, ...,  0.67326184,
        -0.04185125, -0.20930762],
       [-0.26071157, -0.09780911,  0.23047743, ...,  0.11369697,
         0.12302336, -0.04066788]])

### Feature selection 
* train L1 regularizaed regession model to select features
* train RF and select features based on feature importance

In [37]:
# TODO

## Modelling

### Linear regression

In [38]:
# linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_val)

print('Linear Regression train score: ', lin_reg.score(X_train, y_train))
print('Linear Regression test score: ', lin_reg.score(X_val, y_val))

Linear Regression train score:  0.5373065621338003
Linear Regression test score:  0.30975393764797143


### Polynomial regression

In [39]:
# polynomial regression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

poly_reg = Pipeline([
        ("poly_features", PolynomialFeatures(degree=2, include_bias=False)),
        ("lin_reg", LinearRegression()),
    ])
poly_reg.fit(X_train, y_train)
y_pred = poly_reg.predict(X_val)

print('Polynomial Regression train score: ', poly_reg.score(X_train, y_train))
print('Polynomial Regression test score: ', poly_reg.score(X_val, y_val))

Polynomial Regression train score:  0.8603672530310722
Polynomial Regression test score:  -9.28696107443508


### RF

In [40]:
# random forest regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(X_train, y_train)
y_pred = forest_reg.predict(X_val)

print('Random Forest Regression train score: ', forest_reg.score(X_train, y_train))
print('Random Forest Regression test score: ', forest_reg.score(X_val, y_val))

Random Forest Regression train score:  0.9787837790791976
Random Forest Regression test score:  0.8638257843566127


### XGBoost

In [41]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.2,
                max_depth = 4, alpha = 3, n_estimators = 100)
xg_reg.fit(X_train,y_train)
y_pred = xg_reg.predict(X_val)

print('XGBoost Regression train score: ', xg_reg.score(X_train, y_train))
print('XGBoost Regression test score: ', xg_reg.score(X_val, y_val))

XGBoost Regression train score:  0.9720230738193522
XGBoost Regression test score:  0.8768605975271511


### MLP

In [42]:
# mlp
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

mlp_reg = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=100, alpha=0.001,
                        solver='adam', verbose=10, random_state=42, tol=0.000000001)
mlp_reg.fit(X_train, y_train)
y_pred = mlp_reg.predict(X_val)

print('MLP Regression train score: ', mlp_reg.score(X_train, y_train))
print('MLP Regression test score: ', mlp_reg.score(X_val, y_val))


Iteration 1, loss = 9.91208189
Iteration 2, loss = 8.04189867


Iteration 3, loss = 6.43834047
Iteration 4, loss = 4.82230652
Iteration 5, loss = 3.21178204
Iteration 6, loss = 1.90883469
Iteration 7, loss = 1.16006012
Iteration 8, loss = 0.94647237
Iteration 9, loss = 0.83778368
Iteration 10, loss = 0.69315228
Iteration 11, loss = 0.61407167
Iteration 12, loss = 0.52501517
Iteration 13, loss = 0.45433434
Iteration 14, loss = 0.41205426
Iteration 15, loss = 0.39229104
Iteration 16, loss = 0.37547405
Iteration 17, loss = 0.36050975
Iteration 18, loss = 0.34508750
Iteration 19, loss = 0.33528100
Iteration 20, loss = 0.32488736
Iteration 21, loss = 0.31964353
Iteration 22, loss = 0.30986911
Iteration 23, loss = 0.30159637
Iteration 24, loss = 0.29589775
Iteration 25, loss = 0.28774813
Iteration 26, loss = 0.28253546
Iteration 27, loss = 0.27385405
Iteration 28, loss = 0.27240341
Iteration 29, loss = 0.26376806
Iteration 30, loss = 0.25907340
Iteration 31, loss = 0.25368502
Iteration 32, loss = 0.24878771
Iteration 33, loss = 0.24302884
Iteration 34, l

/home/michal/.local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


## Hyperparameter Optimization

In [16]:
# random search
from sklearn.model_selection import RandomizedSearchCV

param_grid_mlp = {
    'hidden_layer_sizes': [(50, 50), (100, 100), (100, 100, 100), (100, 100, 100, 100)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

param_grid_xgb = {
    'n_estimators': [80, 100, 200],
    'learning_rate': [0.1, 0.3, 0.5],
    'max_depth': [3, 5, 6, 9],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]
}

param_grid_forest = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# do search
mlp_reg = MLPRegressor(max_iter=200, random_state=42, tol=0.000000001)
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)
forest_reg = RandomForestRegressor(random_state=42)

mlp_search = RandomizedSearchCV(mlp_reg, param_grid_mlp, n_iter=100, cv=3, verbose=0, n_jobs=-1)
xgb_search = RandomizedSearchCV(xgb_reg, param_grid_xgb, n_iter=100, cv=3, verbose=0, n_jobs=-1)
forest_search = RandomizedSearchCV(forest_reg, param_grid_forest, n_iter=100, cv=3, verbose=0, n_jobs=-1)

mlp_search.fit(X_train, y_train)
xgb_search.fit(X_train, y_train)
forest_search.fit(X_train, y_train)

# get best of these three
mlp_best = mlp_search.best_estimator_
xgb_best = xgb_search.best_estimator_
forest_best = forest_search.best_estimator_

# compare best of these three
print('MLP Regression train score: ', mlp_best.score(X_train, y_train))
print('MLP Regression test score: ', mlp_best.score(X_val, y_val))
print('XGBoost Regression train score: ', xgb_best.score(X_train, y_train))
print('XGBoost Regression test score: ', xgb_best.score(X_val, y_val))
print('Random Forest Regression train score: ', forest_best.score(X_train, y_train))
print('Random Forest Regression test score: ', forest_best.score(X_val, y_val))



/home/michal/.local/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 24 is smaller than n_iter=100. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


/home/michal/.local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/michal/.local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/michal/.local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/michal/.local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/michal/.local/lib/python3.11/site-packages

MLP Regression train score:  0.9457812403215436
MLP Regression test score:  0.8116780080558501
XGBoost Regression train score:  0.9944849568806696
XGBoost Regression test score:  0.8613572269827126
Random Forest Regression train score:  0.9618481967227533
Random Forest Regression test score:  0.869605087526908


In [22]:
# select best model
# TODO: argue why this one is the best
best_model = forest_best

In [23]:
# evaluate on test set
y_pred = best_model.predict(X_test)
print('Test score: ', best_model.score(X_test, y_test))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))


Test score:  0.8296569459512504
Test RMSE:  0.8591884244971202


## Cross validation

In [24]:
# TODO

## Evaluation on forest type classification

In [25]:
# TODO

### Interpretation
* Select the best interpretatable model, look at the coefficients and try to interpret them

In [26]:
# TODO